<a href="https://colab.research.google.com/github/neelkanthrawat/GNN-exercises/blob/main/Assignment_1_Neel's_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pandas as pd
from typing import List

In [2]:
### define a gaussian function in n_star dimn

####

In [3]:
### define two dimensional gaussian distribution

### this gaussian has the ability to infer (evaluate the probability at the given point)
def gaussian_2d(x:np.array,mean:np.array,cov_mat:np.array):
  '''
  This model will evaluate the probabilty at the given x_vector.
  Args:
  '''
  dimn=np.shape(x)[0]
  # x=np.array(x).reshape((1,dimn))
  # mean=np.array(mean).reshape((1,dimn))
  # print("shape(x),shape(mean)")
  # print(np.shape(x), np.shape(mean))
  # note that x is a row vector

  #
  inv_cov=np.linalg.inv(cov_mat)
  x_minus_mu=x-mean
  # print("x_minus_mu:")
  # print(x_minus_mu)

  # argument for the exp
  arg_for_expn=0.5*(x_minus_mu @ inv_cov @ np.transpose(x_minus_mu))[0,0]
  # print("arg_for_expn:");print(arg_for_expn)
  det_cov=np.linalg.det(cov_mat)
  # print("det_cov");print(det_cov)

  # prob
  prob=np.exp(-arg_for_expn) * (1./(2*np.pi*np.sqrt(det_cov)))
  return prob

In [4]:
x=np.array([[0.5,0.5]])
mean=np.array([[1,1]])
cov_mat=np.array([[0.3,0.22],[0.22,0.8]])
check=gaussian_2d(x,mean,cov_mat)
print(check)

0.2363860723405543


### I have a 2D gaussian model at my disposal for the time being. now I need to check how to train it to fit the given dataset

### fitting the gaussian model
###  when we fit the model, we need some loss function to optimize to find the parameters, is'nt it

### My model should have 2 functionalities:
1. inference ability (done temporarily)
2. generation ability

### EM algorithm for training the params for the GMM

In [29]:
def EM_training(mean:np.array,mixing_coeffs:np.array,covs:np.array, dataset_input:np.array):
  '''
  Args:
  mean=
        shape(mean)= (L X dimension_of_the_problem_instance (whether 1D, 2D, 3D and so on))
  mixing_coeffs=
        shape(mixing_coeffs)= (1 X L)
  covs= dict of covariance matrices.
        shape(covs)= L number of entries, one for each of the P_l(x)
  dataset_input: shape would be (num of data set X dimn_of_the_prob)
  '''
  L=np.shape(mean)[0]
  dimn_of_prob=np.shape(dataset_input)[1]
  # define gamma: responsibility
  num_data=np.shape(dataset_input)[0] ### note shape(dataset_input)=(num data X dimn_of_the_problem)
  gamma=np.zeros(shape=(num_data,L))
  num_iters=1 # denoted as T in notes
  ###
  for t in range(0,num_iters):

    ### evaluate the E step
    # update the gamma.
    for idx_over_data in range(0,num_data):
      ### here i need to evaluate the common denominator that will go inside the next loop
      # print("covs[idx_el]")
      # print(covs[0])
      prod_mixing_coeffs_and_gaussian_l=[mixing_coeffs[0,idx_el]*gaussian_2d(x=dataset_input[idx_over_data,:].reshape((1,dimn_of_prob)),mean=mean[idx_el,:].reshape((1,dimn_of_prob)),cov_mat=covs[idx_el]) for idx_el in range(0,L)]
      # print("pil_Pl_fordfnwnf:")
      # print(prod_mixing_coeffs_and_gaussian_l)
      Dr_for_gamma=np.sum(prod_mixing_coeffs_and_gaussian_l)
      gamma[idx_over_data,:]=[prod_mixing_coeffs_and_gaussian_l[k]/Dr_for_gamma for k in range(0,L)]

    print("gamma obtained:");print(gamma)
    ### M-step: Re-estimating the parameters
    N_l=np.sum(gamma,axis=0)# this is the expression sum_{i=1}^{N(all dataset)} gamma_{il} in the lecture notes
    print("N_l is:");print(N_l)
    # shape of N_l = (1XL)
    ### 1. updating means: the expresion is like a wtd sum of the vectors
    ###
    for idx_el in range(0,L):
      print("index el is:");print(idx_el)
      temp_vec=np.array(
          [gamma[idx_over_data,idx_el]*dataset_input[idx_over_data,:]
           for idx_over_data in range(0,num_data)]
          )
      print("temp vec is:");print(temp_vec)
      #update the mean
      # print("np.sum(temp_vec,axis=0):",np.sum(temp_vec,axis=0))
      # print("mean[idx_el,:]=",mean[idx_el,:])
      mean[idx_el,:]=(1./N_l[idx_el])*np.sum(temp_vec,axis=0)
      print("mean matrix right now:");print(mean)
    ### 2. update the covariance matrix:
    ##### formula for updating the cov mat is: sigma_lth=1/N_l[idx_el] * sum_{i=1}^{N(all_dataset)} * gamma_{il}*some_outer_product


    return gamma

In [30]:
# L=4, N=3, dim_of_prob=2
mean=np.array([[1.5,1.25],[0.75,1.25],[1.3,1.7],[1.4,1.63]])
print("mean:");print(mean)
mixing_coeffs=np.array([[0.25,0.25,0.25,0.25]])
covs={0:np.array([[0.02,0.003],[0.003,0.1]]),
      1: np.array([[0.06,0.007],[0.007,0.05]]),
      2:np.array([[0.068,0.003],[0.007,0.05]]),
      3:np.array([[0.06,0.007],[0.017,0.078]])}
dataset_input=np.array([[1.78,1],[1.75,1.3],[1.03,1.08]])
checking_em=EM_training(mean=mean,
                        mixing_coeffs=mixing_coeffs,
                        covs=covs,
                        dataset_input=dataset_input)
# print("gamma obtained:")
# print(checking_em)

mean:
[[1.5  1.25]
 [0.75 1.25]
 [1.3  1.7 ]
 [1.4  1.63]]
gamma obtained:
[[9.18297348e-01 2.95287972e-04 6.13499528e-03 7.52723686e-02]
 [6.55581269e-01 5.90691003e-04 8.22543748e-02 2.61573665e-01]
 [1.10155835e-02 8.15573224e-01 3.47270990e-02 1.38684093e-01]]
N_l is:
[1.5848942  0.8164592  0.12311647 0.47553013]
index el is:
0
temp vec is:
[[1.63456928 0.91829735]
 [1.14726722 0.85225565]
 [0.01134605 0.01189683]]
np.sum(temp_vec,axis=0): [2.79318255 1.78244983]
mean[idx_el,:]= [1.5  1.25]
mean matrix right now:
[[1.76237792 1.12464909]
 [0.75       1.25      ]
 [1.3        1.7       ]
 [1.4        1.63      ]]
index el is:
1
temp vec is:
[[5.25612591e-04 2.95287972e-04]
 [1.03370926e-03 7.67898304e-04]
 [8.40040421e-01 8.80819082e-01]]
np.sum(temp_vec,axis=0): [0.84159974 0.88188227]
mean[idx_el,:]= [0.75 1.25]
mean matrix right now:
[[1.76237792 1.12464909]
 [1.03079216 1.08013023]
 [1.3        1.7       ]
 [1.4        1.63      ]]
index el is:
2
temp vec is:
[[0.01092029 0.0061

In [7]:
a=np.array([[10,20],[1,2],[111,5]])
np.sum(a,axis=0),np.sum(a,axis=1)

(array([122,  27]), array([ 30,   3, 116]))

In [8]:
np.sum(checking_em,axis=0)

array([2.01210489, 0.98789511])

In [9]:
#dataset_input[0,:],dataset_input[0,:].reshape((1,2))

In [10]:
#mean[0,:],mean[0,:].reshape((1,2))

In [11]:
# gaussian_2d(x=dataset_input[0,:].reshape((1,2)),
#             mean=mean[0,:].reshape((1,2)),
#             cov_mat=covs[0])

In [12]:
a=np.array([[1,2,3]]); b=np.array([[10,100,1000]])
(a @ b.T)[0,0]

3210

In [13]:
np.exp(-1400)

0.0